# Building a GPT

In [1]:
# download training data if it doesn't exist

!if ! test -f data/input.txt; then wget -P ./data https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt; fi


In [2]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# encoding and decoding the string based on characters

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda i: ''.join(itos[c] for c in i) # decoder: take a list of integers, output a string
print(encode('hello you'))
print(decode(encode('hello you')))

[46, 43, 50, 50, 53, 1, 63, 53, 59]
hello you


In [6]:
# encode the entire dataset into a torch.Tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [7]:
# 90% train/val split

n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [37]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will be processed in parallel
block_size = 8 # maximum context length for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    index = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+1:i+block_size+1] for i in index])
    return x, y

In [55]:
xb, yb = get_batch('train')

# visualize what is xb, yb
print([decode(i.tolist()) for i in xb])
print([decode(i.tolist()) for i in yb])

['neighbou', 'at the t', 'w in my ', ' embrace']
['eighbour', 't the ti', ' in my b', 'embracem']


In [57]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7103, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [58]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [63]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.465181589126587


In [69]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



QUS:
Bur hin yo t; d otwree d noith il myo autound, f tyowaive be ardecheiBestourers m.
SS:
KEasth of wipigathant itotow?
CLORuene fang'thalt tin!
G st, anrcen g upe, mis d,
F ELOrtado bl stive y f ye tourith'd lletinyould t seatoshe seis tel she.

nas WBamigend w
CE: dousstheth s VJULatuscon fthend t wore may dis tuthit! f I re t l tirtoucofouliof t, nd ceyora may torvee mumun se olkitharecoant the vetus'sbHe nes din
ANonoseererel f lanenomexare, g m, he herd! nea IS:



NRo to h y at t, asous
